In [3]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
import pytorch_lightning as pl
from src.pcd_de_noising import MistNet
from src.pcd_de_noising import PCDDataset, PointCloudDataModule
from torch.utils.data import DataLoader
import os
import torch
import numpy as np
import time

In [4]:
print(torch.__version__)
# 判断是否支持MPS
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(device)

2.1.2.post3
mps


# Configuration

In [5]:
NUMBER_GPUS = 1
DATASET_PATH = "./data"

# Logging

In [6]:
# Create Tensorboard logger
logger = TensorBoardLogger("./log", name="MistNet")

# Model

### Create new model

In [5]:
model = MistNet()

### Load from checkpoint

In [7]:
model = MistNet.load_from_checkpoint(
    "log/MistNet/set-8/checkpoints/epoch=17-step=1908.ckpt")

# Prepare Data

In [8]:
data_module = PointCloudDataModule(
    os.path.join(DATASET_PATH, "train"),
    os.path.join(DATASET_PATH, "val"),
    os.path.join(DATASET_PATH, "test")
)

# Overfit

In [18]:
trainer = pl.Trainer(
    logger=logger,
    overfit_batches=51,
    max_epochs=50,
    log_every_n_steps=1
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [19]:
trainer.fit(model, data_module)


  | Name              | Type                       | Params
-----------------------------------------------------------------
0 | lila1             | LiLaBlock                  | 55.1 K
1 | lila2             | LiLaBlock                  | 804 K 
2 | lila3             | LiLaBlock                  | 2.2 M 
3 | lila4             | LiLaBlock                  | 4.2 M 
4 | dropout           | Dropout2d                  | 0     
5 | lila5             | LiLaBlock                  | 2.0 M 
6 | classifier        | Conv2d                     | 387   
7 | accuracy          | MulticlassAccuracy         | 0     
8 | average_precision | MulticlassAveragePrecision | 0     
-----------------------------------------------------------------
9.3 M     Trainable params
0         Non-trainable params
9.3 M     Total params
37.281    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]Val found 51 files


/Users/xavier/micromamba/envs/pcd-de-noising/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Train found 51 files                                                       


/Users/xavier/micromamba/envs/pcd-de-noising/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:268: You requested to overfit but enabled train dataloader shuffling. We are turning off the train dataloader shuffling for you.
/Users/xavier/micromamba/envs/pcd-de-noising/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 49: 100%|██████████| 13/13 [00:55<00:00,  0.23it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 13/13 [00:55<00:00,  0.23it/s, v_num=0]


# Train

In [9]:
trainer = pl.Trainer(
    logger=logger,
    max_epochs=50,
    log_every_n_steps=1,
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [10]:
trainer.fit(model, data_module)


  | Name              | Type                       | Params
-----------------------------------------------------------------
0 | lila1             | LiLaBlock                  | 72.4 K
1 | lila2             | LiLaBlock                  | 804 K 
2 | lila3             | LiLaBlock                  | 2.2 M 
3 | lila4             | LiLaBlock                  | 4.2 M 
4 | dropout           | Dropout2d                  | 0     
5 | lila5             | LiLaBlock                  | 2.0 M 
6 | classifier        | Conv2d                     | 1.0 K 
7 | accuracy          | MulticlassAccuracy         | 0     
8 | average_precision | MulticlassAveragePrecision | 0     
-----------------------------------------------------------------
9.3 M     Trainable params
0         Non-trainable params
9.3 M     Total params
37.352    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]Val found 413 files


/Users/xavier/micromamba/envs/pcd-de-noising/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Train found 423 files                                                      


/Users/xavier/micromamba/envs/pcd-de-noising/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 18:   0%|          | 0/106 [00:00<?, ?it/s, v_num=0]            

/Users/xavier/micromamba/envs/pcd-de-noising/lib/python3.12/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


# Test

In [10]:
trainer.test(model, datamodule=data_module)

Test found 450 files


/Users/xavier/micromamba/envs/pcd-de-noising/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing DataLoader 0:   0%|          | 0/113 [00:00<?, ?it/s]Logits max: 42.747650146484375 Logits min: -106.54153442382812


/Users/xavier/micromamba/envs/pcd-de-noising/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Average precision score for one or more classes was `nan`. Ignoring these classes in weighted-average
  warnings.warn(*args, **kwargs)  # noqa: B028


Testing DataLoader 0: 100%|██████████| 113/113 [06:54<00:00,  0.27it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_acc_epoch         0.9933481812477112
      test_ap_epoch         0.9466666579246521
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc_epoch': 0.9933481812477112, 'test_ap_epoch': 0.9466666579246521}]

# Measure Inference Time

In [17]:
# 加载需要推断的点云数据并进行推理，然后保存结果
data_module.setup("test")
test_loader = data_module.test_dataloader()
# # 冻结模型: 关闭梯度更新
# model.freeze()
# 评估模式: 该模式会关闭Dropout等层
model.eval()
model.to(device)  # 将模型转移到MPS

reps = len(test_loader)
print(f"Total {reps} data to process")
timings = [0] * len(test_loader)
account = 1430

for i, batch in enumerate(test_loader):
    # 解包batch，并确保所有数据都移至正确的设备
    distance_1 = batch[0].to(device)
    reflectivity_1 = batch[1].to(device)
    distance_2 = batch[2].to(device)
    reflectivity_2 = batch[3].to(device)
    distance_3 = batch[4].to(device)
    reflectivity_3 = batch[5].to(device)

    start_time = time.time()  # 记录开始时间
    with torch.no_grad():
        pred = model(distance_1, reflectivity_1, distance_2,
                     reflectivity_2, distance_3, reflectivity_3)
    pred = pred.cpu().numpy()  # 将预测结果转移到CPU并转为NumPy数组

    predicted_classes = np.argmax(pred, axis=1)

    end_time = time.time()  # 记录结束时间

    elapsed_time = (end_time - start_time)/pred.shape[0]
    timings[i] = elapsed_time  # 保存每次迭代的时间

    # 保存结果到文件
    for j in range(pred.shape[0]):
        np.save(f"./result/{account}.npy", predicted_classes[j])
        account += 1

mean_time = np.mean(timings)
print(f"Mean time per image: {mean_time} seconds")

Test found 2481 files
Total 621 data to process


KeyboardInterrupt: 

In [ ]:
loader = data_module.test_dataloader()
start, end = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
reps = 100
timings = [0] * reps

distance, reflectivity, labels = next(iter(loader))
distance = distance.cuda()
reflectivity = reflectivity.cuda()
labels = labels.cuda()

for rep in range(reps):
    start.record()
    _ = model(distance, reflectivity)
    end.record()

    # Wait for GPU sync
    torch.cuda.synchronize()
    curr_time = start.elapsed_time(end)
    timings[rep] = curr_time / 1000  # ms to s

images_processed = reps * loader.batch_size
mean_time = sum(timings) / images_processed
f"Mean inference time: {mean_time:.2f}s, mean FPS: {1 / mean_time:.2f}"